# INVADE App creator 

## Start point : 
    - app location : root_rep
    - project idea : project_idea
## Agents 
- **BardClient** : text generative model used in code gen , summerize and logical tasks 
    - dependicies : 
        - Google gemini api key
    - public funcions : 
        - bard(text) -> generated_Text  

- **Document** : storge of generated web app related documents (conception docs and others)
    - dependicies : 
        - MongoDB connection 
    - public funcions : 
        - get_disponible_documents() -> list of documents ids
        -  get_document_by_key(id) -> dict(Documument) 
        - save_document(document , key --optional--) 
        - find(key) -> list of finded values  : search in a dict by key
        - get_sub_dict_from_key(key) -> sub dict : search sub document in document

- **Conceptor** : responsible of conceptions and architecting the web app
    - dependicies : 
        - Instance of BardClient agent 
        - Instance of Document agent
    - public functions  :
        - concept() : create a full conception of the app contain : 
            - overview
            - user_roles
            - system_architecture
            - features
            - database_schema
- **Skloton manager**  :  responsable of the interaction with the OS 
    - dependicies : 
        - Instance of llm_client 
    - public functions : 
        - get_project_skeleton() : return dict of project schema 
        - find_path(file_name) -> file path (global)
        - get_file_content(file_path) -> string(file_content) 
        - create_min_app() : init a min app
        - init_env() : init the envirement  ( install dep an others actions)
        - run_batch_code(bat_code_text) -> status ,feedback
        - overwrite_file(file_path, content) 
        - execute_python_file(file_path) ->  status ,feedback

- **Devoloper** : responsable of Code Gen 
    - dependicies : 
        - instance of llm_client
    - public function : they are kinda messy need a reconcept it , but for now it works but not generelized
        

- **Debuger** : responsable of auto correction of codes and runnability of the app
    - dependicies : 
        - instance of llm_client
        - instance of Skloton manager 
    - public functions : 
        - debug() -> status 

- **GitAgent** : responsable of versianage control 
    - dependecies : None
    - public functions : 
        - init_git() : init the control tool
        - commit(message) :  
        - rollback_to_previous_commit() 
        


# IMPORTANT NOTES
- **Config** : Adjust Classes config depend on you envirement 
    - relocate app_with_auth 
    


In [1]:
from Agents.BardClient import BardClient
from Agents.Documents import Documents


root_rep = r'D:\\apps'
project_idea = "camping"
llm_client = BardClient()
documents = Documents()


## Step 1 
Concept the app

In [ ]:
from Agents.Conceptor import Conceptor

conceptor = Conceptor(documents,llm_client,project_idea)
conceptor.concept()

# Step 2 
Retrive Generated Document to continue the process

In [2]:
system_archi = documents.get_document_by_key("archi_document_id")
project_overview = documents.get_document_by_key("overview_document_id")
project_features = documents.get_document_by_key("features_document_id")
database_schema = documents.get_document_by_key("database_schema_document_id")
user_roles = documents.get_document_by_key("user_roles_document_id")

app_name = project_overview["Project Overview Document"]["Application Name"]
stack = system_archi["System Architecture document"]["Technology Stack"]
app_root = root_rep + "\\" + app_name

all_dependencies = documents.find(project_features,"dependencies")

# Step 3 
init the project 

In [3]:
from Agents.Skloton_manager import Skloton_manager

skoloton_manager = Skloton_manager(app_root,llm_client)

In [4]:

code = skoloton_manager.create_min_app()
success, notes = skoloton_manager.init_env(all_dependencies,stack)
print(success)
print(notes)

Directory created at D:\\apps\Adventure_Hub
Contents of 'D:\app_with_auth' moved to 'D:\\apps\Adventure_Hub'
True
start execution
True
The output of the .bat file indicates a successful installation of various Python dependencies and the setup of a virtual environment. Here's a breakdown:

**1. Virtual Environment Setup:**

* **Creating virtual environment...** - Creates a virtual environment to isolate project dependencies.
* **Activating virtual environment...** - Activates the newly created environment.

**2. Upgrading Package Managers:**

* **Upgrading pip, setuptools, and wheel...** - Updates `pip`, `setuptools`, and `wheel` (package management tools) to the latest versions.

**3. Installing Project Dependencies:**

* **Installing Python dependencies...** -  Installs required libraries for the project:
    * **Flask** - Web framework for building web applications.
    * **Flask-WTF** - Extension for Flask providing form handling and CSRF protection.
    * **Flask-Bcrypt** - Extens

# Step 3 
Commit the v 1.0.0

In [ ]:
from Agents.GitAgent import GitAgent


git_agent = GitAgent(app_root)

git_agent.commit("1.0.0")

# Step 4
Dive into The app with Models and Db connection 
- create models and their CRUD tests

In [5]:
from Agents.Devoloper import Devoloper


dev = Devoloper(stack,llm_client)

In [6]:

import re


tables = []
for item in database_schema["Database Schema"]["Tables"] : 
    if item in ["Users","user"]:
        continue  
    tables.append(item)
print(tables)





project_skolton = skoloton_manager.get_project_skeleton() # os function with skloton manager


order = dev.get_ordre_generating_model(database_schema) # base on the relation between tables there is an order of creating 

#develop all models 
# syncro the models created with pre-build blueprint auth
models_path = app_root + "\\Models"
roles_ENUM_filename = "Roles.py"
roles_code = dev.create_role_enum(user_roles,project_skolton)
skoloton_manager.overwrite_file(f"{models_path}\\{roles_ENUM_filename}",roles_code)

super_user_path = models_path + "\\Users.py"
super_user_code = skoloton_manager.get_file_content(super_user_path)

pattern = r"(\w+)\s*="

# Extract all matches (enum names) using re.findall
enum_names = re.findall(pattern, roles_code)

super_user_code = super_user_code.replace(f"RoleEnum.USER.value", f"RoleEnum.{enum_names[0]}.value")

skoloton_manager.overwrite_file(super_user_path,super_user_code)
skoloton_manager.overwrite_file(f"{models_path}//__init__.py"," ")
start = {
    "Table" : "Users",
    "related_tables_name" : []
}
test_order = []
test_order.append(start)
for table in order :
    
    table = table.replace("'","").strip() 
    if table.lower() in ["users","user"]:
        continue  
    related_tables = []
  

    table_details = documents.find(database_schema,table)
    print(table_details)
    related_tables_name = documents.find(table_details,"related Table Name")
    if not related_tables_name : 
        pass
    elif not isinstance(related_tables_name, list) :
       related_tables.append((related_tables_name,documents.find(database_schema,related_tables_name) ))
    else :
        for t in related_tables_name :
            related_tables.append((related_tables_name,documents.find(database_schema,t))) 
    
    table_details= documents.remove_key(table_details,"related Table Name")

    #call the dev to create_model 
    project_skolton = skoloton_manager.get_project_skeleton() # os function with skloton manager
    code = dev.create_model(table,project_skolton,table_details,related_tables)
    print(code)
    file_path = models_path + f"\\{table}.py"
    print(file_path)

    #call the skloton manager to write the code generated
    skoloton_manager.overwrite_file(file_path, code)
    start = {
        "Table" : table,
        "related_tables_name" : related_tables_name
    }
    test_order.append(start)
for test in test_order  :
    table = test["Table"]

    table_path = skoloton_manager.find_path(f"{table}.py")

    table_code = skoloton_manager.get_file_content(table_path)

    related_tables_names = test["related_tables_name"]
    related_code = []
    for related_table in related_tables_names :
        related_table_path = skoloton_manager.find_path(f"{related_table}.py")
        related_table_code = skoloton_manager.get_file_content(related_table_path)
        dict = {
            "file" : f"{related_table}.py",
            "content"  : related_table_code
        }
        related_code.append(dict)
    print("inter to coding")
    test_content = dev.code_crud_test_model(project_skolton,table,table_code ,  related_code )

    file_path = app_root + fr"\\tests\\models_tests\\test_{table}.py"
    print(file_path)
    feed_back = skoloton_manager.overwrite_file(file_path,test_content)
    print(feed_back)
skoloton_manager.overwrite_file(fr"{app_root}//tests//models_tests//__init__.py"," ")


['Activities', 'Guides', 'Bookings', 'Reviews', 'Forums', 'Threads', 'Comments', 'Payments']
[{'Columns': {'guide_id': {'Data Type': 'UUID', 'Constraints': 'Primary Key, Not Null, Unique'}, 'user_id': {'Data Type': 'UUID', 'Constraints': 'Foreign Key, References Users(user_id)'}, 'name': {'Data Type': 'VARCHAR', 'Constraints': 'Not Null'}, 'description': {'Data Type': 'TEXT', 'Constraints': 'Not Null'}, 'specialization': {'Data Type': 'VARCHAR', 'Constraints': 'Not Null'}, 'location': {'Data Type': 'VARCHAR', 'Constraints': 'Not Null'}, 'certification': {'Data Type': 'VARCHAR', 'Constraints': 'Not Null'}, 'image': {'Data Type': 'VARCHAR', 'Constraints': 'URL'}, 'created_at': {'Data Type': 'TIMESTAMP', 'Constraints': 'Not Null, Default Current Timestamp'}, 'updated_at': {'Data Type': 'TIMESTAMP', 'Constraints': 'Not Null, Default Current Timestamp'}}, 'related Table Name': 'Users, Activities, Bookings, Reviews'}]

from mongoengine import Document, UUIDField, StringField, DateTimeField, 

1

In [7]:
skoloton_manager.get_project_skeleton()

{'app.py': None,
 'auth': {'auth_ip_agent_verif_middleware.py': None,
  'auth_permessions.py': None,
  'auth_routes.py': None,
  'auth_schemas.py': None,
  '__init__.py': None},
 'config.py': None,
 'Models': {'Activities.py': None,
  'Bookings.py': None,
  'Comments.py': None,
  'Forums.py': None,
  'Guides.py': None,
  'Payments.py': None,
  'Reviews.py': None,
  'Roles.py': None,
  'Threads.py': None,
  'Users.py': None,
  '__init__.py': None},
 'requirements.txt': None,
 'tests': {'conftest.py': None,
  'models_tests': {'test_Activities.py': None,
   'test_Bookings.py': None,
   'test_Comments.py': None,
   'test_Forums.py': None,
   'test_Guides.py': None,
   'test_Payments.py': None,
   'test_Reviews.py': None,
   'test_Threads.py': None,
   'test_Users.py': None,
   '__init__.py': None},
  '__init__.py': None}}

# Step 5 
Start Debuging app and auto correction 

ps : this step is incomplete and should be fixed or reconcepted

In [5]:
from Agents.Debuger import Debuger

#exemple 
file_name = "app.py"
file_path = skoloton_manager.find_path(file_name)
debuger = Debuger(llm_client,skoloton_manager,stack,file_name,app_root)
status = debuger.debug()


D:\\apps\Adventure_Hub\app.py
EEEEEEEE
ERROR: test_Activities (unittest.loader._FailedTest.test_Activities)
----------------------------------------------------------------------
ImportError: Failed to import test module: test_Activities
Traceback (most recent call last):
  File "c:\Python311\Lib\unittest\loader.py", line 407, in _find_test_path
    module = self._get_module_from_name(name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\unittest\loader.py", line 350, in _get_module_from_name
    __import__(name)
  File "D:\apps\Adventure_Hub\Models\test\test_Activities.py", line 5, in <module>
    from Models import Activities
  File "D:\apps\Adventure_Hub\Models\Activities.py", line 4, in <module>
    class Activities(Document):
  File "D:\apps\Adventure_Hub\Models\Activities.py", line 15, in Activities
    created_at = DateTimeField(required=True, default=datetime.datetime.utcnow)
                                                      ^^^^^^^^
NameError: name '